In [25]:
import pandas as pd
pd.set_option('display.max_columns', 50000)
import numpy as np
import csv
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [26]:
def log1p(vec):
    return np.log1p(abs(vec))

def expm1(x):
    return np.expm1(x)

def clipExp(vec):
    return np.clip(expm1(vec), 0, None)

In [27]:
df = pd.read_csv(filepath_or_buffer="./dataset/train.csv")
df.head()

,Deal_title,Lead_name,Industry,Deal_value,Weighted_amount,Date_of_creation,Pitch,Contact_no,Lead_revenue,Fund_category,Geography,Location,POC_name,Designation,Lead_POC_email,Hiring_candidate_role,Lead_source,Level_of_meeting,Last_lead_update,Internal_POC,Resource,Internal_rating,Success_probability
0,TitleM5DZY,"Davis, Perkins and Bishop Inc",Restaurants,320506$,2067263.7$,2020-03-29,Product_2,607.447.7883,50 - 100 Million,Category 2,USA,"Killeen-Temple, TX",Charlene Werner,Executive Vice President,charlenewerner@davis.com,Community pharmacist,Website,Level 3,No track,"Davis,Sharrice A",NaN,3,73.6
1,TitleKIW18,Bender PLC LLC,Construction Services,39488$,240876.8$,2019-07-10,Product_2,892-938-9493,500 Million - 1 Billion,Category 4,India,Ratlam,rakhi,Chairman/CEO/President,terrylogan@bender.com,Recruitment consultant,Others,Level 1,Did not hear back after Level 1,"Brown,Maxine A",No,5,58.9
2,TitleFXSDN,Carter-Henry and Sons,Hospitals/Clinics,359392$,2407926.4$,2019-07-27,Product_1,538.748.2271,500 Million - 1 Billion,Category 4,USA,"Albany-Schenectady-Troy, NY",Ariel Hamilton,SVP/General Counsel,arielhamilton@carterhenry.com,Health service manager,Marketing Event,Level 1,?,"Georgakopoulos,Vasilios T",No,4,68.8
3,TitlePSK4Y,Garcia Ltd Ltd,Real Estate,76774$,468321.4$,2021-01-30,Product_2,(692)052-1389x75188,500 Million - 1 Billion,Category 3,USA,"Mount Vernon-Anacortes, WA",Erin Wilson,CEO/Co-Founder/Chairman,erinwilson@garcia.com,"Therapist, speech and language",Contact Email,Level 2,Did not hear back after Level 1,"Brown,Maxine A",We have all the requirements,1,64.5
4,Title904GV,Lee and Sons PLC,Financial Services,483896$,NaN,2019-05-22,Product_2,001-878-814-6134x015,50 - 100 Million,Category 3,India,Shimoga,kavita,Executive Vice President,mr.christopher@lee.com,Media planner,Website,Level 2,Up-to-date,"Thomas,Lori E",No,4,62.4


In [28]:
df.describe()

,Internal_rating,Success_probability
count,7007.000000,7007.000000
mean,3.009562,64.745133
std,1.418666,17.931635
min,1.000000,-5.000000
25%,2.000000,60.600000
50%,3.000000,65.300000
75%,4.000000,69.600000
max,5.000000,107.340000


In [29]:
df.dtypes

Deal_title                object
Lead_name                 object
Industry                  object
Deal_value                object
Weighted_amount           object
Date_of_creation          object
Pitch                     object
Contact_no                object
Lead_revenue              object
Fund_category             object
Geography                 object
Location                  object
POC_name                  object
Designation               object
Lead_POC_email            object
Hiring_candidate_role     object
Lead_source               object
Level_of_meeting          object
Last_lead_update          object
Internal_POC              object
Resource                  object
Internal_rating            int64
Success_probability      float64
dtype: object

In [30]:
def dropInitialRedundantFeatures(df):
    redundantColumns = ["Lead_name","Contact_no","Location","POC_name","Lead_POC_email","Internal_POC"]
    df = df.drop(redundantColumns,axis=1)
    return df
df = dropInitialRedundantFeatures(df)

In [31]:
def handleDesignation(df):
    
    def myDesignationPreProcess(designation):
        
        if designation == "Chief Executive Officer":
            return "CEO"
        elif designation == "Chairman/CEO/President":
            return "CEO/Chairman/President"
        else:
            return designation
    
    df["Designation"] = df.apply(lambda row : myDesignationPreProcess(row["Designation"]),axis=1)
    
    return df
df = handleDesignation(df)

In [32]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import string
from nltk.stem import PorterStemmer

    
corpus = ""

for ind in df.index:
    currentRowValue = df.loc[ind,"Hiring_candidate_role"]
    if len(currentRowValue.split(" ")) > 1:
        currentRowValue = currentRowValue.lower().replace("manager","")
        currentRowValue = currentRowValue.lower().replace("officer","")
        currentRowValue = currentRowValue.lower().replace("engineer","")

    corpus = corpus + " "+ currentRowValue

def preProcessCorpus(hiringCandidateString):

    hiringCandidateString = hiringCandidateString.lower()

    hiringCandidateString = hiringCandidateString.replace("/"," ")
    hiringCandidateString = hiringCandidateString.replace("'s","")

    # collecting a list of stop words from nltk and punctuation form
    # string class and create single array.
    stopset = stopwords.words('english') + list(string.punctuation)
    # remove stop words and punctuations from string.
    # word_tokenize is used to tokenize the input corpus in word tokens.
    hiringCandidateString = " ".join([i for i in word_tokenize(hiringCandidateString) if i not in stopset])
    # remove non-ascii characters
    hiringCandidateString = unidecode(hiringCandidateString)

    ps = PorterStemmer()
    stringList = [ps.stem(tok) for tok in word_tokenize(hiringCandidateString)]
    hiringCandidateString = ' '.join(stringList)

    return hiringCandidateString

corpus = preProcessCorpus(corpus)

original_list = word_tokenize(corpus)

def hiringCandidateRoleHandling(df):

    
    def getFrequency(processedWord, dictionaryList):
        try:
            frequencyOfWord = pd.Series(dictionaryList).value_counts()[processedWord]
            return frequencyOfWord
        except:
            return 0
        
    def getHighestFrequencyWord(processedSentence):
        
        tokens = word_tokenize(processedSentence)
        
        tokFrequencyList = []
        for tok in tokens:
            tokFrequency = getFrequency(tok,original_list)
            tokFrequencyList.append(tokFrequency)
            
        highestFrequencyIndex = tokFrequencyList.index(max(tokFrequencyList))
        
        return tokens[highestFrequencyIndex]
        
    
    def getGeneralRole(rowValue):
        
        rowValue = preProcessCorpus(rowValue)
        
        return getHighestFrequencyWord(rowValue)
        
        
    df["Hiring_candidate_role"] = df.apply(lambda row : getGeneralRole(row["Hiring_candidate_role"]),axis=1)
    
    return df
df = hiringCandidateRoleHandling(df)
df.head()

,Deal_title,Industry,Deal_value,Weighted_amount,Date_of_creation,Pitch,Lead_revenue,Fund_category,Geography,Designation,Hiring_candidate_role,Lead_source,Level_of_meeting,Last_lead_update,Resource,Internal_rating,Success_probability
0,TitleM5DZY,Restaurants,320506$,2067263.7$,2020-03-29,Product_2,50 - 100 Million,Category 2,USA,Executive Vice President,commun,Website,Level 3,No track,NaN,3,73.6
1,TitleKIW18,Construction Services,39488$,240876.8$,2019-07-10,Product_2,500 Million - 1 Billion,Category 4,India,CEO/Chairman/President,consult,Others,Level 1,Did not hear back after Level 1,No,5,58.9
2,TitleFXSDN,Hospitals/Clinics,359392$,2407926.4$,2019-07-27,Product_1,500 Million - 1 Billion,Category 4,USA,SVP/General Counsel,health,Marketing Event,Level 1,?,No,4,68.8
3,TitlePSK4Y,Real Estate,76774$,468321.4$,2021-01-30,Product_2,500 Million - 1 Billion,Category 3,USA,CEO/Co-Founder/Chairman,therapist,Contact Email,Level 2,Did not hear back after Level 1,We have all the requirements,1,64.5
4,Title904GV,Financial Services,483896$,NaN,2019-05-22,Product_2,50 - 100 Million,Category 3,India,Executive Vice President,planner,Website,Level 2,Up-to-date,No,4,62.4


In [33]:
def handleAmountColumns(df):
    
    def myAmountPreProcess(amount):
        if not pd.isnull(amount) or not pd.isna(amount):
            amount = amount.replace("$", "")
            
        return amount
        
    df["Deal_value"] = df.apply(lambda row : myAmountPreProcess(row["Deal_value"]),axis=1)
    
    df["Weighted_amount"] = df.apply(lambda row : myAmountPreProcess(row["Weighted_amount"]),axis=1)
    
    
    df["Deal_value"] = pd.to_numeric(df["Deal_value"])
    df["Weighted_amount"] = pd.to_numeric(df["Weighted_amount"])
    
    df["Deal_value"] = df["Deal_value"].fillna(df["Deal_value"].median())
    df["Weighted_amount"] = df["Weighted_amount"].fillna(df["Weighted_amount"].median())
    
    return df
df = handleAmountColumns(df)

In [34]:
def handleDatesColumns(df):
    
    df['Date_of_creation']  = pd.to_datetime(df['Date_of_creation'],format="%Y-%m-%d")
    
    df["creation_year"] = df["Date_of_creation"].dt.year
    
    df = df.drop(["Date_of_creation"],axis=1)
    
    return df

df = handleDatesColumns(df)

In [35]:
df.head()

,Deal_title,Industry,Deal_value,Weighted_amount,Pitch,Lead_revenue,Fund_category,Geography,Designation,Hiring_candidate_role,Lead_source,Level_of_meeting,Last_lead_update,Resource,Internal_rating,Success_probability,creation_year
0,TitleM5DZY,Restaurants,320506.0,2067263.70,Product_2,50 - 100 Million,Category 2,USA,Executive Vice President,commun,Website,Level 3,No track,NaN,3,73.6,2020
1,TitleKIW18,Construction Services,39488.0,240876.80,Product_2,500 Million - 1 Billion,Category 4,India,CEO/Chairman/President,consult,Others,Level 1,Did not hear back after Level 1,No,5,58.9,2019
2,TitleFXSDN,Hospitals/Clinics,359392.0,2407926.40,Product_1,500 Million - 1 Billion,Category 4,USA,SVP/General Counsel,health,Marketing Event,Level 1,?,No,4,68.8,2019
3,TitlePSK4Y,Real Estate,76774.0,468321.40,Product_2,500 Million - 1 Billion,Category 3,USA,CEO/Co-Founder/Chairman,therapist,Contact Email,Level 2,Did not hear back after Level 1,We have all the requirements,1,64.5,2021
4,Title904GV,Financial Services,483896.0,1548238.95,Product_2,50 - 100 Million,Category 3,India,Executive Vice President,planner,Website,Level 2,Up-to-date,No,4,62.4,2019


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7007 entries, 0 to 7006
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Deal_title             7007 non-null   object 
 1   Industry               7006 non-null   object 
 2   Deal_value             7007 non-null   float64
 3   Weighted_amount        7007 non-null   float64
 4   Pitch                  7007 non-null   object 
 5   Lead_revenue           7007 non-null   object 
 6   Fund_category          7007 non-null   object 
 7   Geography              6035 non-null   object 
 8   Designation            7007 non-null   object 
 9   Hiring_candidate_role  7007 non-null   object 
 10  Lead_source            7007 non-null   object 
 11  Level_of_meeting       7007 non-null   object 
 12  Last_lead_update       6374 non-null   object 
 13  Resource               6858 non-null   object 
 14  Internal_rating        7007 non-null   int64  
 15  Succ

In [37]:
print(df["Industry"].value_counts())
print("-"*50)

print(df["Pitch"].value_counts())
print("-"*50)

print(df["Lead_revenue"].value_counts())
print("-"*50)

print(df["Fund_category"].value_counts())
print("-"*50)

print(df["Geography"].value_counts())
print("-"*50)

print(df["Lead_source"].value_counts())
print("-"*50)

print(df["Level_of_meeting"].value_counts())
print("-"*50)

print(df["Last_lead_update"].value_counts())
print("-"*50)

print(df["Resource"].value_counts())
print("-"*50)

print(df["Designation"].value_counts())
print("-"*50)

print(df["Hiring_candidate_role"].value_counts())
print("-"*50)

Banks                        1480
Insurance                     439
Financial Services            397
Real Estate                   209
Investment Bank/Brokerage     200
                             ... 
Hardware Consulting             1
Newspapers                      1
Auto Dealers                    1
Multimedia Hardware             1
Speech Recognition              1
Name: Industry, Length: 171, dtype: int64
--------------------------------------------------
Product_1    3504
Product_2    3503
Name: Pitch, dtype: int64
--------------------------------------------------
500 Million - 1 Billion    2377
100 - 500 Million          2351
50 - 100 Million           2279
Name: Lead_revenue, dtype: int64
--------------------------------------------------
Category 3    1791
Category 2    1785
Category 1    1745
Category 4    1686
Name: Fund_category, dtype: int64
--------------------------------------------------
USA      3061
India    2974
Name: Geography, dtype: int64
---------------------

In [38]:
def handleMissingValues(df):
    
    df["Industry"] = df["Industry"].fillna(df["Industry"].mode()[0]) # most occurred value
    
    df["Resource"] = df["Resource"].fillna(df["Resource"].mode()[0]) # most occurred value
    
    df["Last_lead_update"] = df["Last_lead_update"].fillna(df["Last_lead_update"].mode()[0]) # most occurred value
        
    df["Geography"] = df["Geography"].fillna(df["Geography"].mode()[0]) # most occurred value
    
    return df
df = handleMissingValues(df)

In [39]:
df.isnull().sum()

Deal_title               0
Industry                 0
Deal_value               0
Weighted_amount          0
Pitch                    0
Lead_revenue             0
Fund_category            0
Geography                0
Designation              0
Hiring_candidate_role    0
Lead_source              0
Level_of_meeting         0
Last_lead_update         0
Resource                 0
Internal_rating          0
Success_probability      0
creation_year            0
dtype: int64

In [40]:
# def labelEncoding(df, onlyTransform: False, label_object = {}):
#     categorical_columns = [
#         'Industry',
#         'Lead_revenue',
#         'Last_lead_update'
#     ]
#     for col in categorical_columns:
#         labelencoder = None
#         if onlyTransform is False:
#             labelencoder = LabelEncoder()
#             labelencoder.fit(df[col])
#             label_object[col] = labelencoder
#         else:
#             labelencoder = label_object[col]
            
#         df[col] = labelencoder.transform(df[col])
#     return df, label_object

# df, label_object = labelEncoding(df, onlyTransform= False, label_object= {})
# df.head()

In [41]:
def oneHotEncoding(df, onlyTransform=False, one_hot_object = {}):
    categorical_columns = [
        "Industry",
        "Pitch",
        "Lead_revenue",
        "Fund_category",
        "Geography",
        "Lead_source",
        "Level_of_meeting",
        "Last_lead_update",
        "Resource",
        "Designation",
        "Hiring_candidate_role",
        "creation_year"
    ]
    for col in categorical_columns:
        
        col_data  = df[col].values.reshape(-1,1)
        
        if onlyTransform is False:
            onehot_encoder = OneHotEncoder()
            onehot_encoder = onehot_encoder.fit(col_data)
            one_hot_object[col] = onehot_encoder
        
        onehot_encoded = one_hot_object[col].transform(col_data).toarray()
        columns_list = []
        for i in range(onehot_encoded.shape[1]):
            columns_list.append(col+"_"+str(int(i)))
        dfOneHot = pd.DataFrame(onehot_encoded, columns = columns_list) 
        df = pd.concat([df, dfOneHot], axis=1)
        

    df = df.drop(categorical_columns,axis=1)
    return df, one_hot_object


df, one_hot_object = oneHotEncoding(df, onlyTransform=False, one_hot_object = {})
df.head()

,Deal_title,Deal_value,Weighted_amount,Internal_rating,Success_probability,Industry_0,Industry_1,Industry_2,Industry_3,Industry_4,Industry_5,Industry_6,Industry_7,Industry_8,Industry_9,Industry_10,Industry_11,Industry_12,Industry_13,Industry_14,Industry_15,Industry_16,Industry_17,Industry_18,Industry_19,Industry_20,Industry_21,Industry_22,Industry_23,Industry_24,Industry_25,Industry_26,Industry_27,Industry_28,Industry_29,Industry_30,Industry_31,Industry_32,Industry_33,Industry_34,Industry_35,Industry_36,Industry_37,Industry_38,Industry_39,Industry_40,Industry_41,Industry_42,Industry_43,Industry_44,Industry_45,Industry_46,Industry_47,Industry_48,Industry_49,Industry_50,Industry_51,Industry_52,Industry_53,Industry_54,Industry_55,Industry_56,Industry_57,Industry_58,Industry_59,Industry_60,Industry_61,Industry_62,Industry_63,Industry_64,Industry_65,Industry_66,Industry_67,Industry_68,Industry_69,Industry_70,Industry_71,Industry_72,Industry_73,Industry_74,Industry_75,Industry_76,Industry_77,Industry_78,Industry_79,Industry_80,Industry_81,Industry_82,Industry_83,Industry_84,Industry_85,Industry_86,Industry_87,Industry_88,Industry_89,Industry_90,Industry_91,Industry_92,Industry_93,Industry_94,Industry_95,Industry_96,Industry_97,Industry_98,Industry_99,Industry_100,Industry_101,Industry_102,Industry_103,Industry_104,Industry_105,Industry_106,Industry_107,Industry_108,Industry_109,Industry_110,Industry_111,Industry_112,Industry_113,Industry_114,Industry_115,Industry_116,Industry_117,Industry_118,Industry_119,Industry_120,Industry_121,Industry_122,Industry_123,Industry_124,Industry_125,Industry_126,Industry_127,Industry_128,Industry_129,Industry_130,Industry_131,Industry_132,Industry_133,Industry_134,Industry_135,Industry_136,Industry_137,Industry_138,Industry_139,Industry_140,Industry_141,Industry_142,Industry_143,Industry_144,Industry_145,Industry_146,Industry_147,Industry_148,Industry_149,Industry_150,Industry_151,Industry_152,Industry_153,Industry_154,Industry_155,Industry_156,Industry_157,Industry_158,Industry_159,Industry_160,Industry_161,Industry_162,Industry_163,Industry_164,Industry_165,Industry_166,Industry_167,Industry_168,Industry_169,Industry_170,Pitch_0,Pitch_1,Lead_revenue_0,Lead_revenue_1,Lead_revenue_2,Fund_category_0,Fund_category_1,Fund_category_2,Fund_category_3,Geography_0,Geography_1,Lead_source_0,Lead_source_1,Lead_source_2,Lead_source_3,Level_of_meeting_0,Level_of_meeting_1,Level_of_meeting_2,Last_lead_update_0,Last_lead_update_1,Last_lead_update_2,Last_lead_update_3,Last_lead_update_4,Last_lead_update_5,Last_lead_update_6,Last_lead_update_7,Last_lead_update_8,Last_lead_update_9,Last_lead_update_10,Resource_0,Resource_1,Resource_2,Resource_3,Resource_4,Resource_5,Designation_0,Designation_1,Designation_2,Designation_3,Designation_4,Designation_5,Designation_6,Designation_7,Hiring_candidate_role_0,Hiring_candidate_role_1,Hiring_candidate_role_2,Hiring_candidate_role_3,Hiring_candidate_role_4,Hiring_candidate_role_5,Hiring_candidate_role_6,Hiring_candidate_role_7,Hiring_candidate_role_8,Hiring_candidate_role_9,Hiring_candidate_role_10,Hiring_candidate_role_11,Hiring_candidate_role_12,Hiring_candidate_role_13,Hiring_candidate_role_14,Hiring_candidate_role_15,Hiring_candidate_role_16,Hiring_candidate_role_17,Hiring_candidate_role_18,Hiring_candidate_role_19,Hiring_candidate_role_20,Hiring_candidate_role_21,Hiring_candidate_role_22,Hiring_candidate_role_23,Hiring_candidate_role_24,Hiring_candidate_role_25,Hiring_candidate_role_26,Hiring_candidate_role_27,Hiring_candidate_role_28,Hiring_candidate_role_29,Hiring_candidate_role_30,Hiring_candidate_role_31,Hiring_candidate_role_32,Hiring_candidate_role_33,Hiring_candidate_role_34,Hiring_candidate_role_35,Hiring_candidate_role_36,Hiring_candidate_role_37,Hiring_candidate_role_38,Hiring_candidate_role_39,Hiring_candidate_role_40,Hiring_candidate_role_41,Hiring_candidate_role_42,Hiring_candidate_role_43,Hiring_candidate_role_44,Hiring_candidate_role_45,Hiring_candidate_role_

In [42]:
# df["Success_probability"] = log1p(df["Success_probability"])
def clipPercentages(val):
    val = abs(val)
    if val > 100:
        return 100
    else:
        return val
df["Success_probability"] = df.apply(lambda row : clipPercentages(row["Success_probability"]),axis=1)

y = pd.DataFrame(df["Success_probability"])
df = df.drop("Success_probability", axis=1)

df = df.drop("Deal_title",axis=1)

X = pd.DataFrame(df)
print(X.shape)
print(y.shape)

(7007, 435)
(7007, 1)


In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)


X_train = X
y_train = y

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(7007, 435)
(7007, 1)
(701, 435)
(701, 1)


In [44]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler



scalerY = MinMaxScaler()
scalerY = scalerY.fit(y_train)
y_train = scalerY.transform(y_train)
y_train = pd.DataFrame(y_train)

y_test = scalerY.transform(y_test)
y_test = pd.DataFrame(y_test)




scalerX = MinMaxScaler()
scalerX = scalerX.fit(X_train)
X_train = scalerX.transform(X_train)
X_train = pd.DataFrame(X_train)

X_test = scalerX.transform(X_test)
X_test = pd.DataFrame(X_test)

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2, L1

model = Sequential()

model.add(Dense(32,input_dim=X_train.shape[1], activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(128, kernel_regularizer=L1(0.005), activation='relu'))

model.add(Dense(64, activation='relu'))

# model.add(Dense(32, kernel_regularizer=L1(0.01), activation='relu'))

# model.add(Dense(16,activation='relu'))

model.add(Dense(1,activation='linear'))

adam = Adam(learning_rate=0.0001)

model.compile(optimizer=adam,
                loss='mse')

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32)                13952     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_9 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 32,705
Trainable params: 32,705
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434
0,0.639895,0.572982,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.076110,0.064622,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [47]:
import os
import datetime
import tensorflow as tf
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [48]:
model.fit(
    X_train,
    y_train,
#     validation_data=(X_test,y_test),
    batch_size=100,
    epochs=250,
    verbose=1,
    callbacks=[tensorboard_callback])

Epoch 1/250
71/71 [==============================] - 1s 11ms/step - loss: 3.8299
Epoch 2/250
71/71 [==============================] - 0s 4ms/step - loss: 3.3259
Epoch 3/250
71/71 [==============================] - 0s 4ms/step - loss: 3.0510
Epoch 4/250
71/71 [==============================] - 0s 4ms/step - loss: 2.7967
Epoch 5/250
71/71 [==============================] - 0s 4ms/step - loss: 2.5534
Epoch 6/250
71/71 [==============================] - 0s 4ms/step - loss: 2.3217
Epoch 7/250
71/71 [==============================] - 0s 4ms/step - loss: 2.1027
Epoch 8/250
71/71 [==============================] - 0s 4ms/step - loss: 1.8912
Epoch 9/250
71/71 [==============================] - 0s 4ms/step - loss: 1.6928
Epoch 10/250
71/71 [==============================] - 0s 4ms/step - loss: 1.5079
Epoch 11/250
71/71 [==============================] - 0s 4ms/step - loss: 1.3308
Epoch 12/250
71/71 [==============================] - 0s 4ms/step - loss: 1.1647
Epoch 13/250
71/71 [================

71/71 [==============================] - 0s 4ms/step - loss: 0.0291
Epoch 103/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0273
Epoch 104/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0260
Epoch 105/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0262
Epoch 106/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0276
Epoch 107/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0281
Epoch 108/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0264
Epoch 109/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0269
Epoch 110/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0276
Epoch 111/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0259
Epoch 112/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0272
Epoch 113/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0274
Epoch 114/250
71/71 [=========

71/71 [==============================] - 0s 4ms/step - loss: 0.0186
Epoch 202/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0186
Epoch 203/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0193
Epoch 204/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0182
Epoch 205/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0184
Epoch 206/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0185
Epoch 207/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0186
Epoch 208/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0183
Epoch 209/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0186
Epoch 210/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0184
Epoch 211/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0183
Epoch 212/250
71/71 [==============================] - 0s 4ms/step - loss: 0.0179
Epoch 213/250
71/71 [=========

In [49]:
import sklearn.metrics as metrics
val_prediction = model.predict(X_test)
val_prediction = scalerY.inverse_transform(val_prediction)
# val_prediction = expm1(val_prediction)


val_original_y = scalerY.inverse_transform(y_test)
# val_original_y = expm1(val_original_y)


score = max(0, 100-np.sqrt(metrics.mean_squared_error(val_original_y, val_prediction)))
score

88.60373551132487

In [50]:
df_test = pd.read_csv("./dataset/test.csv")
df_test.head()

,Deal_title,Lead_name,Industry,Deal_value,Weighted_amount,Date_of_creation,Pitch,Contact_no,Lead_revenue,Fund_category,Geography,Location,POC_name,Designation,Lead_POC_email,Hiring_candidate_role,Lead_source,Level_of_meeting,Last_lead_update,Internal_POC,Resource,Internal_rating
0,TitleAD16O,Bonilla Ltd Inc,Investment Bank/Brokerage,200988$,NaN,2020-04-15,Product_1,167.332.2751x989,100 - 500 Million,Category 4,India,Bhubaneshwar,sonia,Chairman/CEO/President,maureenthomas@bonilla.com,"Designer, fashion/clothing",Marketing Event,Level 1,more than a month,"Massiah,Gerard F",No,-1.0
1,TitleOW6CR,"Williams, Rogers and Roach PLC",Electronics,409961$,2541758.2$,2021-01-23,Product_1,001-486-903-0711x7831,100 - 500 Million,Category 3,USA,"Coeur d'Alene, ID",Daniel Bell,CEO/Co-Founder/Chairman,danielbell@williams.com,Horticultural consultant,Marketing Event,Level 2,Up-to-date,"Smith,Keenan H",Yes,1.0
2,TitleVVJQ5,"Wood, Vaughn and Morales Ltd",Banks,434433$,3041031.0$,2020-07-19,Product_1,(393)104-2610x9723,100 - 500 Million,Category 1,USA,"Portland-South Portland, ME",Andrew Davis,Chairman/Chief Innovation Officer,andrewdavis@wood.com,Information officer,Marketing Event,Level 2,Did not hear back after Level 1,"Gilley,Janine",Deliverable,5.0
3,TitleUS8NA,Durham-Crawford Inc,Music,218952$,1521716.4$,2020-02-27,Product_2,(817)040-4599,100 - 500 Million,Category 1,India,Bareilly,shital,CEO/Chairman/President,charlesrivera@durhamcrawford.com,Commercial/residential surveyor,Contact Email,Level 3,more than a month,"Morsy,Omar A",No,5.0
4,Title5VGWW,"Simpson, Duncan and Long LLC",Real Estate,392835$,2455218.75$,2020-10-25,Product_1,718-032-5726x76098,500 Million - 1 Billion,Category 3,USA,"Trenton, NJ",Shelly Stephenson,CEO/Co-Founder/Chairman,shellystephenson@simpson.com,Wellsite geologist,Others,Level 3,More than 2 weeks,"Morsy,Omar A",Deliverable,2.0


In [51]:
df_test = dropInitialRedundantFeatures(df_test)

In [52]:
df_test = handleDesignation(df_test)

In [53]:
df_test = hiringCandidateRoleHandling(df_test)

In [54]:
df_test = handleAmountColumns(df_test)

In [55]:
df_test = handleDatesColumns(df_test)

In [56]:
df_test = handleMissingValues(df_test)

In [57]:
df_test, one_hot_object = oneHotEncoding(df_test, onlyTransform=True, one_hot_object = one_hot_object)

In [58]:
df_test.head()

,Deal_title,Deal_value,Weighted_amount,Internal_rating,Industry_0,Industry_1,Industry_2,Industry_3,Industry_4,Industry_5,Industry_6,Industry_7,Industry_8,Industry_9,Industry_10,Industry_11,Industry_12,Industry_13,Industry_14,Industry_15,Industry_16,Industry_17,Industry_18,Industry_19,Industry_20,Industry_21,Industry_22,Industry_23,Industry_24,Industry_25,Industry_26,Industry_27,Industry_28,Industry_29,Industry_30,Industry_31,Industry_32,Industry_33,Industry_34,Industry_35,Industry_36,Industry_37,Industry_38,Industry_39,Industry_40,Industry_41,Industry_42,Industry_43,Industry_44,Industry_45,Industry_46,Industry_47,Industry_48,Industry_49,Industry_50,Industry_51,Industry_52,Industry_53,Industry_54,Industry_55,Industry_56,Industry_57,Industry_58,Industry_59,Industry_60,Industry_61,Industry_62,Industry_63,Industry_64,Industry_65,Industry_66,Industry_67,Industry_68,Industry_69,Industry_70,Industry_71,Industry_72,Industry_73,Industry_74,Industry_75,Industry_76,Industry_77,Industry_78,Industry_79,Industry_80,Industry_81,Industry_82,Industry_83,Industry_84,Industry_85,Industry_86,Industry_87,Industry_88,Industry_89,Industry_90,Industry_91,Industry_92,Industry_93,Industry_94,Industry_95,Industry_96,Industry_97,Industry_98,Industry_99,Industry_100,Industry_101,Industry_102,Industry_103,Industry_104,Industry_105,Industry_106,Industry_107,Industry_108,Industry_109,Industry_110,Industry_111,Industry_112,Industry_113,Industry_114,Industry_115,Industry_116,Industry_117,Industry_118,Industry_119,Industry_120,Industry_121,Industry_122,Industry_123,Industry_124,Industry_125,Industry_126,Industry_127,Industry_128,Industry_129,Industry_130,Industry_131,Industry_132,Industry_133,Industry_134,Industry_135,Industry_136,Industry_137,Industry_138,Industry_139,Industry_140,Industry_141,Industry_142,Industry_143,Industry_144,Industry_145,Industry_146,Industry_147,Industry_148,Industry_149,Industry_150,Industry_151,Industry_152,Industry_153,Industry_154,Industry_155,Industry_156,Industry_157,Industry_158,Industry_159,Industry_160,Industry_161,Industry_162,Industry_163,Industry_164,Industry_165,Industry_166,Industry_167,Industry_168,Industry_169,Industry_170,Pitch_0,Pitch_1,Lead_revenue_0,Lead_revenue_1,Lead_revenue_2,Fund_category_0,Fund_category_1,Fund_category_2,Fund_category_3,Geography_0,Geography_1,Lead_source_0,Lead_source_1,Lead_source_2,Lead_source_3,Level_of_meeting_0,Level_of_meeting_1,Level_of_meeting_2,Last_lead_update_0,Last_lead_update_1,Last_lead_update_2,Last_lead_update_3,Last_lead_update_4,Last_lead_update_5,Last_lead_update_6,Last_lead_update_7,Last_lead_update_8,Last_lead_update_9,Last_lead_update_10,Resource_0,Resource_1,Resource_2,Resource_3,Resource_4,Resource_5,Designation_0,Designation_1,Designation_2,Designation_3,Designation_4,Designation_5,Designation_6,Designation_7,Hiring_candidate_role_0,Hiring_candidate_role_1,Hiring_candidate_role_2,Hiring_candidate_role_3,Hiring_candidate_role_4,Hiring_candidate_role_5,Hiring_candidate_role_6,Hiring_candidate_role_7,Hiring_candidate_role_8,Hiring_candidate_role_9,Hiring_candidate_role_10,Hiring_candidate_role_11,Hiring_candidate_role_12,Hiring_candidate_role_13,Hiring_candidate_role_14,Hiring_candidate_role_15,Hiring_candidate_role_16,Hiring_candidate_role_17,Hiring_candidate_role_18,Hiring_candidate_role_19,Hiring_candidate_role_20,Hiring_candidate_role_21,Hiring_candidate_role_22,Hiring_candidate_role_23,Hiring_candidate_role_24,Hiring_candidate_role_25,Hiring_candidate_role_26,Hiring_candidate_role_27,Hiring_candidate_role_28,Hiring_candidate_role_29,Hiring_candidate_role_30,Hiring_candidate_role_31,Hiring_candidate_role_32,Hiring_candidate_role_33,Hiring_candidate_role_34,Hiring_candidate_role_35,Hiring_candidate_role_36,Hiring_candidate_role_37,Hiring_candidate_role_38,Hiring_candidate_role_39,Hiring_candidate_role_40,Hiring_candidate_role_41,Hiring_candidate_role_42,Hiring_candidate_role_43,Hiring_candidate_role_44,Hiring_candidate_role_45,Hiring_candidate_role_46,Hiring_candidate_

In [59]:
Deal_title = df_test["Deal_title"]
Deal_title_df = pd.DataFrame(data = Deal_title,columns=["Deal_title"])
df_test = df_test.drop(["Deal_title"],axis=1)
df_test.head()

,Deal_value,Weighted_amount,Internal_rating,Industry_0,Industry_1,Industry_2,Industry_3,Industry_4,Industry_5,Industry_6,Industry_7,Industry_8,Industry_9,Industry_10,Industry_11,Industry_12,Industry_13,Industry_14,Industry_15,Industry_16,Industry_17,Industry_18,Industry_19,Industry_20,Industry_21,Industry_22,Industry_23,Industry_24,Industry_25,Industry_26,Industry_27,Industry_28,Industry_29,Industry_30,Industry_31,Industry_32,Industry_33,Industry_34,Industry_35,Industry_36,Industry_37,Industry_38,Industry_39,Industry_40,Industry_41,Industry_42,Industry_43,Industry_44,Industry_45,Industry_46,Industry_47,Industry_48,Industry_49,Industry_50,Industry_51,Industry_52,Industry_53,Industry_54,Industry_55,Industry_56,Industry_57,Industry_58,Industry_59,Industry_60,Industry_61,Industry_62,Industry_63,Industry_64,Industry_65,Industry_66,Industry_67,Industry_68,Industry_69,Industry_70,Industry_71,Industry_72,Industry_73,Industry_74,Industry_75,Industry_76,Industry_77,Industry_78,Industry_79,Industry_80,Industry_81,Industry_82,Industry_83,Industry_84,Industry_85,Industry_86,Industry_87,Industry_88,Industry_89,Industry_90,Industry_91,Industry_92,Industry_93,Industry_94,Industry_95,Industry_96,Industry_97,Industry_98,Industry_99,Industry_100,Industry_101,Industry_102,Industry_103,Industry_104,Industry_105,Industry_106,Industry_107,Industry_108,Industry_109,Industry_110,Industry_111,Industry_112,Industry_113,Industry_114,Industry_115,Industry_116,Industry_117,Industry_118,Industry_119,Industry_120,Industry_121,Industry_122,Industry_123,Industry_124,Industry_125,Industry_126,Industry_127,Industry_128,Industry_129,Industry_130,Industry_131,Industry_132,Industry_133,Industry_134,Industry_135,Industry_136,Industry_137,Industry_138,Industry_139,Industry_140,Industry_141,Industry_142,Industry_143,Industry_144,Industry_145,Industry_146,Industry_147,Industry_148,Industry_149,Industry_150,Industry_151,Industry_152,Industry_153,Industry_154,Industry_155,Industry_156,Industry_157,Industry_158,Industry_159,Industry_160,Industry_161,Industry_162,Industry_163,Industry_164,Industry_165,Industry_166,Industry_167,Industry_168,Industry_169,Industry_170,Pitch_0,Pitch_1,Lead_revenue_0,Lead_revenue_1,Lead_revenue_2,Fund_category_0,Fund_category_1,Fund_category_2,Fund_category_3,Geography_0,Geography_1,Lead_source_0,Lead_source_1,Lead_source_2,Lead_source_3,Level_of_meeting_0,Level_of_meeting_1,Level_of_meeting_2,Last_lead_update_0,Last_lead_update_1,Last_lead_update_2,Last_lead_update_3,Last_lead_update_4,Last_lead_update_5,Last_lead_update_6,Last_lead_update_7,Last_lead_update_8,Last_lead_update_9,Last_lead_update_10,Resource_0,Resource_1,Resource_2,Resource_3,Resource_4,Resource_5,Designation_0,Designation_1,Designation_2,Designation_3,Designation_4,Designation_5,Designation_6,Designation_7,Hiring_candidate_role_0,Hiring_candidate_role_1,Hiring_candidate_role_2,Hiring_candidate_role_3,Hiring_candidate_role_4,Hiring_candidate_role_5,Hiring_candidate_role_6,Hiring_candidate_role_7,Hiring_candidate_role_8,Hiring_candidate_role_9,Hiring_candidate_role_10,Hiring_candidate_role_11,Hiring_candidate_role_12,Hiring_candidate_role_13,Hiring_candidate_role_14,Hiring_candidate_role_15,Hiring_candidate_role_16,Hiring_candidate_role_17,Hiring_candidate_role_18,Hiring_candidate_role_19,Hiring_candidate_role_20,Hiring_candidate_role_21,Hiring_candidate_role_22,Hiring_candidate_role_23,Hiring_candidate_role_24,Hiring_candidate_role_25,Hiring_candidate_role_26,Hiring_candidate_role_27,Hiring_candidate_role_28,Hiring_candidate_role_29,Hiring_candidate_role_30,Hiring_candidate_role_31,Hiring_candidate_role_32,Hiring_candidate_role_33,Hiring_candidate_role_34,Hiring_candidate_role_35,Hiring_candidate_role_36,Hiring_candidate_role_37,Hiring_candidate_role_38,Hiring_candidate_role_39,Hiring_candidate_role_40,Hiring_candidate_role_41,Hiring_candidate_role_42,Hiring_candidate_role_43,Hiring_candidate_role_44,Hiring_candidate_role_45,Hiring_candidate_role_46,Hiring_candidate_role_47,Hir

In [60]:
x_test_submit = scalerX.transform(df_test)
x_test_submit = pd.DataFrame(x_test_submit)
x_test_submit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434
0,0.400115,0.433301,-0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.819362,0.705053,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [61]:
prediction = model.predict(x_test_submit)
prediction = scalerY.inverse_transform(prediction)

In [62]:
df_to_submit = pd.concat([Deal_title_df,pd.DataFrame(data = prediction,columns=["Success_probability"])], axis=1)

In [63]:
# df_to_submit["Success_probability"] = expm1(df_to_submit["Success_probability"])
df_to_submit.head()

,Deal_title,Success_probability
0,TitleAD16O,53.728016
1,TitleOW6CR,69.502495
2,TitleVVJQ5,63.458424
3,TitleUS8NA,75.002632
4,Title5VGWW,67.433243


In [64]:
df_to_submit.to_csv(path_or_buf="./df_to_submit"+".csv",float_format='%.1f',index=False)